In [5]:
import os
import subprocess
import numpy as np
import pandas as pd
from ete3 import Tree
import random
import glob
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set random seed
random.seed(191919)

# Function to compare trees using TreeCmp
def compare_trees(method, htPath, mtPath, random_trees_path, outPath):
    try:
        # Make comparison folder if it doesn't exist
        compare_folder = os.path.join(outPath, f'compare_{method}')
        if not os.path.isdir(compare_folder):
            os.makedirs(compare_folder)
        
        # Compare host and microbiome tree
        mt_compare_file = os.path.join(compare_folder, 'compare_ht_mt.txt')
        command = f'java -jar "C:/Users/aleja/Downloads/TreeCmp_v1.0-b291/TreeCmp/bin/TreeCmp.jar" -r "{htPath}" -d {method} -i "{mtPath}" -o "{mt_compare_file}" -N'
        logger.info(f'Running command: {command}')
        
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        logger.info(f'Command output: {result.stdout}')
        logger.error(f'Command error: {result.stderr}')
        
        # Check if the file was created
        if not os.path.isfile(mt_compare_file):
            raise FileNotFoundError(f"Expected output file {mt_compare_file} not found")
        
        # Compare host and random trees
        rt_compare_file = os.path.join(compare_folder, 'compare_ht_random.txt')
        command = f'java -jar "C:/Users/aleja/Downloads/TreeCmp_v1.0-b291/TreeCmp/bin/TreeCmp.jar" -r "{htPath}" -d {method} -i "{random_trees_path}" -o "{rt_compare_file}" -N'
        logger.info(f'Running command: {command}')
        
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        logger.info(f'Command output: {result.stdout}')
        logger.error(f'Command error: {result.stderr}')
        
        # Check if the file was created
        if not os.path.isfile(rt_compare_file):
            raise FileNotFoundError(f"Expected output file {rt_compare_file} not found")
        
        # Read comparison results
        mtCompare = pd.read_csv(mt_compare_file, sep='\t')
        rtCompare = pd.read_csv(rt_compare_file, sep='\t')
        
        # Print results
        methods = {"rc":'Rooted Robinson-Foulds', "mc":'Rooted Matching Cluster', "ms":'Unrooted Matching Split', "rf":'Unrooted Robinson-Foulds'}
        logger.info(f' --- METHOD: {methods[method]} --- ')
        logger.info(f"Host-Microbe Score:   {mtCompare.iloc[0,1]}")
        max_stochastic_metric = max(rtCompare.iloc[:,1])
        logger.info(f"Max Stochastic Metric: {max_stochastic_metric}")
        normalized_score = mtCompare.iloc[0,1] / max_stochastic_metric
        logger.info(f'Normalized Score: {normalized_score}')
        congruent_trees_count = len(rtCompare[rtCompare.iloc[:,1] <= mtCompare.iloc[0,1]])
        logger.info(f'Random Trees with Equivalent or More Congruent Score: {congruent_trees_count}')
        total_trees = len(rtCompare)
        logger.info(f'Total Trees: {total_trees}')
        p_value = congruent_trees_count / total_trees
        logger.info(f'P-Value: {p_value}\n')
        
    except Exception as e:
        logger.error(f'Error comparing trees with method {method}: {e}')

# Main script
def main():
    try:
        # User-defined paths
        outPath = r'C:/Users/aleja/OneDrive - Karolinska Institutet/Documentos/JOBS/Liquenes/Lichens_project/Sticta/co-divergence/Reviewed'
        htPath = os.path.join(outPath, 'ht.newick')
        mtPath = os.path.join(outPath, 'mt.newick')
        random_trees_path = os.path.join(outPath, 'random_trees.newick')
        
        # Import trees
        htIn = Tree(htPath)
        mtIn = Tree(mtPath)
        
        # Check if trees have the same number of leaves
        if len(htIn) != len(mtIn):
            raise ValueError('Host and Microbiome Tree Have Different Numbers of Nodes!')
        
        # List to store host leaf names
        htLeafs = [leaf.name for leaf in mtIn]
        
        # Generate random trees
        rtPath = os.path.join(outPath, 'random_trees')
        if not os.path.isdir(rtPath):
            os.makedirs(rtPath)
        
        # Define number of random trees
        numRandom = 10000
        
        for curRandom in np.arange(numRandom):
            t = Tree()
            random.shuffle(htLeafs)
            t.populate(len(htIn), names_library=htLeafs)
            t.write(outfile=os.path.join(rtPath, f'tree_{curRandom}.newick'))
        
        # Concatenate random trees into a single file
        random_tree_files = glob.glob(os.path.join(rtPath, '*'))
        with open(random_trees_path, 'w') as outfile:
            for file in random_tree_files:
                with open(file, 'r') as infile:
                    outfile.write(infile.read())
        
        # Perform comparisons for each method
        for method in ["rc", "mc", "rf", "ms"]:
            compare_trees(method, htPath, mtPath, random_trees_path, outPath)
    
    except Exception as e:
        logger.error(f'Error in main script: {e}')

if __name__ == "__main__":
    main()


INFO:__main__:Running command: java -jar "C:/Users/aleja/Downloads/TreeCmp_v1.0-b291/TreeCmp/bin/TreeCmp.jar" -r "C:/Users/aleja/OneDrive - Karolinska Institutet/Documentos/JOBS/Liquenes/Lichens_project/Sticta/co-divergence/Reviewed\ht.newick" -d rc -i "C:/Users/aleja/OneDrive - Karolinska Institutet/Documentos/JOBS/Liquenes/Lichens_project/Sticta/co-divergence/Reviewed\mt.newick" -o "C:/Users/aleja/OneDrive - Karolinska Institutet/Documentos/JOBS/Liquenes/Lichens_project/Sticta/co-divergence/Reviewed\compare_rc\compare_ht_mt.txt" -N
INFO:__main__:Command output: TreeCmp version 1.0-b291

Active options:
Type of the analysis:  one-to-all comparison mode (-r)
Metrics:
  1. R-F_Cluster (rc)
Input file: C:/Users/aleja/OneDrive - Karolinska Institutet/Documentos/JOBS/Liquenes/Lichens_project/Sticta/co-divergence/Reviewed\mt.newick
Output file: C:/Users/aleja/OneDrive - Karolinska Institutet/Documentos/JOBS/Liquenes/Lichens_project/Sticta/co-divergence/Reviewed\compare_rc\compare_ht_mt.txt
